# modkit extract full -h

Transform the probabilities from the MM/ML tags in a modBAM into a table
Usage: modkit extract full [OPTIONS] <IN_BAM> <OUT_PATH>

Arguments:
  <IN_BAM>    Path to modBAM file to extract read-level information from, or one
              of `-` or `stdin` to specify a stream from standard input. If a
              file is used it may be sorted and have associated index
  <OUT_PATH>  Path to output file, "stdout" or "-" will direct output to
              standard out

Options:
      --reference <REFERENCE>  Path to reference FASTA to extract reference
                               context information from. Required for motif
                               selection
  -h, --help                   Print help (see more with '--help')

Compute Options:
  -t, --threads <THREADS>
          Number of threads to use [default: 4]
      --out-threads <OUT_THREADS>
          Number of threads to use for parallel bgzf writing [default: 4]
  -q, --queue-size <QUEUE_SIZE>
          Number of reads that can be in memory at a time. Increasing this value
          will increase thread usage, at the cost of memory usage [default:
          10000]

Output Options:
      --bgzf                   Write output as BGZF compressed file
      --force                  Force overwrite of output file
      --kmer-size <KMER_SIZE>  Set the query and reference k-mer size (if a
                               reference is provided). Maximum number for this
                               value is 50 [default: 5]
      --no-headers             Don't print the header lines in the output tables

Logging Options:
      --log-filepath <LOG_FILEPATH>  Path to file to write run log

Selection Options:
      --mapped-only
          Include only mapped bases in output (alias: mapped)
      --allow-non-primary
          Output aligned secondary and supplementary base modification
          probabilities as additional rows. The primary alignment will have all
          of the base modification probabilities (including soft-clipped ones,
          unless --mapped-only is used). The non-primary alignments will only
          have mapped bases in the output
      --num-reads <NUM_READS>
          Number of reads to use. Note that when using a sorted, indexed modBAM
          that the sampling algorithm will attempt to sample records evenly over
          the length of the reference sequence. The result is the final number
          of records used may be slightly more or less than the requested
          number. When piping from stdin or using a modBAM without an index, the
          requested number of reads will be the first `num_reads` records
      --region <REGION>
          Process only reads that are aligned to a specified region of the BAM.
          Format should be <chrom_name>:<start>-<end> or <chrom_name>
      --include-bed <INCLUDE_BED>
          BED file with regions to include (alias: include-positions).
          Implicitly only includes mapped sites
  -v, --exclude-bed <EXCLUDE_BED>
          BED file with regions to _exclude_ (alias: exclude)
      --edge-filter <EDGE_FILTER>
          Discard base modification calls that are this many bases from the
          start or the end of the read. Two comma-separated values may be
          provided to asymmetrically filter out base modification calls from the
          start and end of the reads. For example, 4,8 will filter out base
          modification calls in the first 4 and last 8 bases of the read
      --invert-edge-filter
          Invert the edge filter, instead of filtering out base modification
          calls at the ends of reads, only _keep_ base modification calls at the
          ends of reads. E.g. if usually, "4,8" would remove (i.e. filter out)
          base modification calls in the first 4 and last 8 bases of the read,
          using this flag will keep only base modification calls in the first 4
          and last 8 bases

Modified Base Options:
      --motif <MOTIF> <MOTIF>  Output read-level base modification probabilities
                               restricted to the reference sequence motifs
                               provided. The first argument should be the
                               sequence motif and the second argument is the
                               0-based offset to the base to pileup base
                               modification counts for. For example: --motif
                               CGCG 0 indicates include base modifications for
                               which the read is aligned to the first C on the
                               top strand and the last C (complement to G) on
                               the bottom strand. The --cpg argument is short
                               hand for --motif CG 0. This argument can be
                               passed multiple times
      --annotate-motifs        When used with `--motif` or `--cpg` emit all
                               modified base alignment information even if it
                               does not align to a reference motif, but annotate
                               which aligned positions match which motifs in the
                               "motifs" column. "." will be used when an aligned
                               position does not match a motif
      --cpg                    Only output counts at CpG motifs. Requires a
                               reference sequence to be provided


## We do modkit extract calls with filter-threshold and cpg just to get CpG methylated C with a threshold of more than 0.80. Then we generate a python script that cound the number of times that each read appear on this file and generate another .tsv file with the amount of m for each read on the region of the BED file. Pay attention to the fact that cutted reads will have fewer m, but for this is important to keep the right cut site on the reference.

modkit extract calls \
./data/IE20_multiplex/IE20_barcode03/IE20_barcode03_lowfiltered_alignedcutq4.bam \
./output/IE20_multiplex/IE20_barcode03/modkit_extractcalls/IE20_barcode03_q4/IE20_barcode03_extractcalls_3000q4.tsv \
--reference ./data/reference_genomes/cut_hg002v1.1.fasta \
--include-bed ./data/reference_regions/CpG_island_subtelomeres_3000.bed \
--filter-threshold C:0.80 \
--cpg 


# modkit extract calls -h
Produce a table of read-level base modification calls. This table has, for each
read, one row for each base modification call in that read using the same
thresholding algorithm as in pileup, or summary (see online documentation for
details on thresholds)

Usage: modkit extract calls [OPTIONS] <IN_BAM> <OUT_PATH>

Arguments:
  <IN_BAM>    Path to modBAM file to extract read-level information from, or one
              of `-` or `stdin` to specify a stream from standard input. If a
              file is used it may be sorted and have associated index
  <OUT_PATH>  Path to output file, "stdout" or "-" will direct output to
              standard out

Options:
      --reference <REFERENCE>  Path to reference FASTA to extract reference
                               context information from. If no reference is
                               provided, `ref_kmer` column will be "." in the
                               output. (alias: ref)
  -h, --help                   Print help (see more with '--help')

Compute Options:
  -t, --threads <THREADS>
          Number of threads to use [default: 4]
      --out-threads <OUT_THREADS>
          Number of threads to use for parallel bgzf writing [default: 4]
  -q, --queue-size <QUEUE_SIZE>
          Number of reads that can be in memory at a time. Increasing this value
          will increase thread usage, at the cost of memory usage [default:
          10000]

Output Options:
      --bgzf                   Write output as BGZF compressed file
      --force                  Force overwrite of output file
      --kmer-size <KMER_SIZE>  Set the query and reference k-mer size (if a
                               reference is provided). Maximum number for this
                               value is 50 [default: 5]
      --no-headers             Don't print the header lines in the output tables

Logging Options:
      --log-filepath <LOG_FILEPATH>  Path to file to write run log

Selection Options:
      --mapped-only
          Include only mapped bases in output (alias: mapped)
      --allow-non-primary
          Output aligned secondary and supplementary base modification
          probabilities as additional rows. The primary alignment will have all
          of the base modification probabilities (including soft-clipped ones,
          unless --mapped-only is used). The non-primary alignments will only
          have mapped bases in the output
      --num-reads <NUM_READS>
          Number of reads to use. Note that when using a sorted, indexed modBAM
          that the sampling algorithm will attempt to sample records evenly over
          the length of the reference sequence. The result is the final number
          of records used may be slightly more or less than the requested
          number. When piping from stdin or using a modBAM without an index, the
          requested number of reads will be the first `num_reads` records
      --region <REGION>
          Process only reads that are aligned to a specified region of the BAM.
          Format should be <chrom_name>:<start>-<end> or <chrom_name>
      --include-bed <INCLUDE_BED>
          BED file with regions to include (alias: include-positions).
          Implicitly only includes mapped sites
  -v, --exclude-bed <EXCLUDE_BED>
          BED file with regions to _exclude_ (alias: exclude)
      --edge-filter <EDGE_FILTER>
          Discard base modification calls that are this many bases from the
          start or the end of the read. Two comma-separated values may be
          provided to asymmetrically filter out base modification calls from the
          start and end of the reads. For example, 4,8 will filter out base
          modification calls in the first 4 and last 8 bases of the read
      --invert-edge-filter
          Invert the edge filter, instead of filtering out base modification
          calls at the ends of reads, only _keep_ base modification calls at the
          ends of reads. E.g. if usually, "4,8" would remove (i.e. filter out)
          base modification calls in the first 4 and last 8 bases of the read,
          using this flag will keep only base modification calls in the first 4
          and last 8 bases
      --pass-only
          Only output base modification calls that pass the minimum confidence
          threshold. (alias: pass)

Modified Base Options:
      --motif <MOTIF> <MOTIF>  Output read-level base modification probabilities
                               restricted to the reference sequence motifs
                               provided. The first argument should be the
                               sequence motif and the second argument is the
                               0-based offset to the base to pileup base
                               modification counts for. For example: --motif
                               CGCG 0 indicates include base modifications for
                               which the read is aligned to the first C on the
                               top strand and the last C (complement to G) on
                               the bottom strand. The --cpg argument is short
                               hand for --motif CG 0. This argument can be
                               passed multiple times
      --annotate-motifs        When used with `--motif` or `--cpg` emit all
                               modified base alignment information even if it
                               does not align to a reference motif, but annotate
                               which aligned positions match which motifs in the
                               "motifs" column. "." will be used when an aligned
                               position does not match a motif
      --cpg                    Only output counts at CpG motifs. Requires a
                               reference sequence to be provided

Filtering Options:
      --filter-threshold <FILTER_THRESHOLD>
          Specify the filter threshold globally or per-base. Global filter
          threshold can be specified with by a decimal number (e.g. 0.75).
          Per-base thresholds can be specified by colon-separated values, for
          example C:0.75 specifies a threshold value of 0.75 for cytosine
          modification calls. Additional per-base thresholds can be specified by
          repeating the option: for example --filter-threshold C:0.75
          --filter-threshold A:0.70 or specify a single base option and a
          default for all other bases with: --filter-threshold A:0.70
          --filter-threshold 0.9 will specify a threshold value of 0.70 for
          adenine and 0.9 for all other base modification calls

Sampling Options:
  -n, --sample-num-reads <SAMPLE_NUM_READS>
          Sample this many reads when estimating the filtering threshold. If a
          sorted, indexed modBAM is provided reads will be sampled evenly across
          aligned genome. If a region is specified, with the --region, then
          reads will be sampled evenly across the region given. This option is
          useful for large BAM files. In practice, 10-50 thousand reads is
          sufficient to estimate the model output distribution and determine the
          filtering threshold [default: 10042]
Transform the probabilities from the MM/ML tags in a modBAM into a table


# modkit pileup -h

modkit pileup path/to/reads.bam output/path/pileup.bed --ref path/to/reference.fasta

Tabulates base modification calls across genomic positions. This command
produces a bedMethyl formatted file. Schema and description of fields can be
found in the README

Usage: modkit pileup [OPTIONS] <IN_BAM> <OUT_BED>

Arguments:
  <IN_BAM>   Input BAM, should be sorted and have associated index available
  <OUT_BED>  Output file (or directory with --bedgraph option) to write results
             into. Specify "-" or "stdout" to direct output to stdout

Options:
      --preset <PRESET>     Optional preset options for specific applications.
                            traditional: Prepares bedMethyl analogous to that
                            generated from other technologies for the analysis
                            of 5mC modified bases. Shorthand for --cpg
                            --combine-strands --ignore h [possible values:
                            traditional]
      --invert-edge-filter  Invert the edge filter, instead of filtering out
                            base modification calls at the ends of reads, only
                            _keep_ base modification calls at the ends of reads.
                            E.g. if usually, "4,8" would remove (i.e. filter
                            out) base modification calls in the first 4 and last
                            8 bases of the read, using this flag will keep only
                            base modification calls in the first 4 and last 8
                            bases
  -h, --help                Print help (see more with '--help')

Logging Options:
      --log-filepath <LOG_FILEPATH>
          Specify a file for debug logs to be written to, otherwise ignore them.
          Setting a file is recommended. (alias: log)

Selection Options:
      --region <REGION>  Process only the specified region of the BAM when
                         performing pileup. Format should be
                         <chrom_name>:<start>-<end> or <chrom_name>. Commas are
                         allowed

Compute Options:
  -t, --threads <THREADS>  Number of threads to use while processing chunks
                           concurrently [default: 4]

Sampling Options:
  -n, --num-reads <NUM_READS>  Sample this many reads when estimating the
                               filtering threshold. Reads will be sampled evenly
                               across aligned genome. If a region is specified,
                               either with the --region option or the
                               --sample-region option, then reads will be
                               sampled evenly across the region given. This
                               option is useful for large BAM files. In
                               practice, 10-50 thousand reads is sufficient to
                               estimate the model output distribution and
                               determine the filtering threshold [default:
                               10042]

Filtering Options:
      --no-filtering
          Do not perform any filtering, include all mod base calls in output.
          See filtering.md for details on filtering
      --filter-threshold <FILTER_THRESHOLD>
          Specify the filter threshold globally or per-base. Global filter
          threshold can be specified with by a decimal number (e.g. 0.75).
          Per-base thresholds can be specified by colon-separated values, for
          example C:0.75 specifies a threshold value of 0.75 for cytosine
          modification calls. Additional per-base thresholds can be specified by
          repeating the option: for example --filter-threshold C:0.75
          --filter-threshold A:0.70 or specify a single base option and a
          default for all other bases with: --filter-threshold A:0.70
          --filter-threshold 0.9 will specify a threshold value of 0.70 for
          adenine and 0.9 for all other base modification calls
      --mod-thresholds <MOD_THRESHOLDS>
          Specify a passing threshold to use for a base modification,
          independent of the threshold for the primary sequence base or the
          default. For example, to set the pass threshold for 5hmC to 0.8 use
          `--mod-threshold h:0.8`. The pass threshold will still be estimated as
          usual and used for canonical cytosine and other modifications unless
          the `--filter-threshold` option is also passed. See the online
          documentation for more details
      --sample-region <SAMPLE_REGION>
          Specify a region for sampling reads from when estimating the threshold
          probability. If this option is not provided, but --region is provided,
          the genomic interval passed to --region will be used. Format should be
          <chrom_name>:<start>-<end> or <chrom_name>

Modified Base Options:
      --motif <MOTIF> <MOTIF>  Output pileup counts for only sequence motifs
                               provided. The first argument should be the
                               sequence motif and the second argument is the
                               0-based offset to the base to pileup base
                               modification counts for. For example: --motif
                               CGCG 0 indicates to pileup counts for the first C
                               on the top strand and the last C (complement to
                               G) on the bottom strand. The --cpg argument is
                               short hand for --motif CG 0
      --cpg                    Only output counts at CpG motifs. Requires a
                               reference sequence to be provided as well as FAI
                               index
  -r, --ref <REFERENCE_FASTA>  Reference sequence in FASTA format. Required for
                               motif (e.g. CpG) filtering, requires FAI fasta
                               index to be pre-generated
      --combine-strands        When performing motif analysis (such as CpG), sum
                               the counts from the positive and negative strands
                               into the counts for the positive strand position

Output Options:
      --header
          Output a header with the bedMethyl
      --prefix <PREFIX>
          Prefix to prepend on bedgraph output file names. Without this option
          the files will be <mod_code>_<strand>.bedgraph
      --partition-tag <PARTITION_TAG>
          Partition output into multiple bedMethyl files based on tag-value
          pairs. The output will be multiple bedMethyl files with the format
          `<prefix>_<tag_value_1>_<tag_value_2>_<tag_value_n>.bed` prefix is
          optional and set with the `--prefix` flag

## To achieve the number of methylations per read we did:

modkit extract calls \
./data/IE20_multiplex/IE20_barcode03/IE20_barcode03_lowfiltered_alignedcut.bam \
./output/IE20_multiplex/IE20_barcode03/IE20_barcode03_CpG_island_subtelomeres_6000.tsv \
--reference ./data/reference_genomes/cut_hg002v1.1.fasta \
--include-bed ./data/reference_regions/CpG_island_subtelomeres_6000.bed \
--filter-threshold C:0.80 \
--cpg 

## Now we use a python script to sum all m calls on those regions and generate another file, with the read_id, number of m, and end site. At some point we should also calculate the non-methylated cpg to the a %methylation on the region. I think we can do this with another python script.

bgzip pipelog.bed

modkit stats \
/storage/scratch01/groups/bu/teloseq/modkit/data/reference_regions/IE20_barcode03_chrreference.bed.gz \
--regions /storage/scratch01/groups/bu/teloseq/modkit/data/reference_regions/CpG_island_subtelomeres_3000.bed \
--out-table IE20_barcode03_CpGislands_bamreference_stats.tsv
> bedmethyl does not have contig chr14_MATERNAL_P, skipping.
> bedmethyl does not have contig chr15_MATERNAL_P, skipping.
> bedmethyl does not have contig chr15_MATERNAL_Q, skipping.
> bedmethyl does not have contig chr15_PATERNAL_P, skipping.
> bedmethyl does not have contig chr16_PATERNAL_Q, skipping.
> bedmethyl does not have contig chr18_PATERNAL_Q, skipping.
> bedmethyl does not have contig chr19_PATERNAL_Q, skipping.
> bedmethyl does not have contig chr20_MATERNAL_P, skipping.
> bedmethyl does not have contig chr20_PATERNAL_P, skipping.
> bedmethyl does not have contig chr21_PATERNAL_Q, skipping.
> bedmethyl does not have contig chr6_MATERNAL_P, skipping.
> bedmethyl does not have contig chr7_PATERNAL_P, skipping.
> bedmethyl does not have contig chr8_PATERNAL_Q, skipping.
> bedmethyl does not have contig chrY_PATERNAL_P, skipping.

## The chromosomes that have been skipped are because no reads are mapped there. YYYYY genialll, ya tengo el número de count_valid_m per read en esas regiones para cada chromosome :)
#chrom                  start   end     name    strand  count_a count_valid_a   percent_a       count_h count_valid_h   percent_h       count_m count_valid_m   percent_m
chr10_MATERNAL_P        21221   24221   .       .       24      3063            0.7835455       26      2816            0.92329544      2309    2816            81.995735
chr10_MATERNAL_Q        27175   30175   .       .       173     9862            1.7542081       24      7449            0.3221909       2853    7449            38.30044
chr10_PATERNAL_P        20352   23352   .       .       78      14957           0.521495        242     23979           1.0092164       21339   23979           88.990364


## To sum all the m that are per read
cd /storage/scratch01/groups/bu/teloseq/modkit/output/IE20_multiplex/IE20_barcode03/modkit_extractcalls
python count_m_perread.py IE20_barcode03_CpG_island_subtelomeres_6000.tsv IE20_barcode03_CpG_island_subtelomeres_6000_perread.tsv


## To count how many CG motif are on the reference so thus how many are we expecting to calculate the percentage

python count_CG.py /storage/scratch01/groups/bu/teloseq/modkit/data/reference_genomes/cut_hg002v1.1.fasta /storage/scratch01/groups/bu/teloseq/modkit/data/reference_regions/CpG_island_subtelomeres_3000_hIPSC.bed CpG_island_subtelomeres_3000_counts.tsv

# Esto nos da el fichero .tsv con el número de CG motif que hay en el genoma de reference cut_hgv002 para la region de window the 3000 base pair
